In [49]:
import tensorflow as tf
import pandas as pd
print(tf.__version__)

1.13.1


In [128]:
df = pd.read_csv("/Users/bryanwu/Desktop/taxi_fare_prediction/new-york-city-taxi-fare-prediction/train.csv", nrows=10000, parse_dates=['pickup_datetime'])




In [129]:
df = df.drop(["key"], 1)
pickup_datetime = df.pop('pickup_datetime')
mean = df.mean()
std = df.std()
df = (df - df.mean())/df.std()
df['dayofweek'] = pickup_datetime.dt.dayofweek

In [130]:
target = df.pop('fare_amount')
train_input_fn = tf.estimator.inputs.pandas_input_fn(x=df, y=target, batch_size=16, num_epochs=50, shuffle=True, target_column="fare_amount")

In [67]:
def parse(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, FIELD_DEFAULTS)
    
    features = dict(zip(COLUMN_NAMES,fields))
    
    #transform date(named key) to day of week
    #pick_up_date = features.pop('day')
    #print(pick_up_date)
    #features["day"] = pd.to_datetime(pd.Series(pick_up_date)).dt.dayofweek
    
    #drop date and pass_count
    features.pop("day")
    features.pop("passenger_count")
    features.pop("pickup_datetime")
    # Separate the label from the features
    label = features.pop('fare_amount')
    print(features)
    return features, label


In [131]:
feature_columns = [tf.feature_column.numeric_column(column_name, default_value=0.0) for column_name in df.columns]
feature_columns

[NumericColumn(key='pickup_longitude', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='pickup_latitude', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='dropoff_longitude', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='dropoff_latitude', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='passenger_count', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='dayofweek', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None)]

In [132]:
est = tf.estimator.LinearRegressor(feature_columns=feature_columns)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wt/dv0qlxcn5j9b0h9vd8tr23bh0000gn/T/tmpbftoixvl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb2bab10b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [133]:
def input_fn_train():
    train_path = "/Users/bryanwu/Desktop/taxi_fare_prediction/new-york-city-taxi-fare-prediction/train.csv"
    dataset = tf.data.TextLineDataset(train_path).skip(1)
    dataset = dataset.map(parse).batch(10)
    return dataset


In [134]:
est.train(input_fn=train_input_fn)


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/wt/dv0qlxcn5j9b0h9vd8tr23bh0000gn/T/tmpbftoixvl/model.ckpt.
INFO:tensorflow:loss = 47.90061, step = 1
INFO:tensorflow:global_step/sec: 121.571
INFO:tensorflow:loss = 21.025085, step = 101 (0.832 sec)
INFO:tensorflow:global_step/sec: 148.617
INFO:tensorflow:loss = 5.960574, step = 201 (0.667 sec)
INFO:tensorflow:global_step/sec: 124.029
INFO:tensorflow:loss = 1.7390919, step = 301 (0.810 sec)
INFO:tensorflow:global_step/sec: 144.916
INFO:tensorflow:los

KeyboardInterrupt: 